In [ ]:
# !pip install underthesea
# !pip install demoji
# !pip install pyvi

In [ ]:
# !apt update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
# !tar -xvf spark-3.3.0-bin-hadoop3.tgz
# !pip install -q findspark
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)

# %cd '/content/gdrive/My Drive/Data_Sicience/LDS0_K282_ONLINE_LuuNguyenKhaiHoan/Project_1/Data/'

In [ ]:
# jupyter contrib nbextension install

In [ ]:
%autosave 60

In [1]:
%cd "Data"

C:\Users\khaih\Downloads\LDS0\Data


In [2]:
import pandas as pd
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize
import demoji
from pyvi import ViPosTagger, ViTokenizer
import string

In [3]:
# %cd '/content/gdrive/My Drive/Data_Sicience/LDS0_K282_ONLINE_LuuNguyenKhaiHoan/Project_1/Data/'

In [4]:
import findspark
findspark.init()

In [5]:
import pyspark

In [6]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer, VectorIndexer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import LogisticRegression, LinearSVC
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.sql.functions import col, lit
import pyspark.sql.functions as F
from pyspark.sql.functions import *
import regex
from pyspark.mllib.evaluation import MulticlassMetrics

In [7]:
#SparkContext.setSystemProperty('spark.executor.memory', '12g')
conf = SparkConf()
conf.set('spark.executor.memory', '12g')
conf.set('spark.driver.memory', '20g')
conf.set('spark.cores.max', '16')
conf.set('spark.network.timeout', '60')
sc = SparkContext.getOrCreate(conf)

In [8]:
spark=SparkSession(sc)

In [9]:
import pandas as pd
data = pd.read_csv("pre_data.csv", delimiter=',',index_col=0)

In [10]:
data.head()

,class,pre_comment
259179,Like,kiểm không
259180,Like,cho_phép sơ_suất hát hoạt_động rõ_ràng saisĩ
259181,Like,vừa_vặn không nâu dày đỏ trắng đỏ cổ náu đẹp n...
259182,Like,đầu_shop hợp
259183,Like,đẹp đẹp hàng thích


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509691 entries, 259179 to 1319048
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   class        509691 non-null  object
 1   pre_comment  509691 non-null  object
dtypes: object(2)
memory usage: 11.7+ MB


In [12]:
# data.to_csv("pre_data_1.csv")

In [13]:
stringcols = data.select_dtypes(include='object').columns
data[stringcols] = data[stringcols].astype("str")

In [14]:
data[stringcols]

,class,pre_comment
259179,Like,kiểm không
259180,Like,cho_phép sơ_suất hát hoạt_động rõ_ràng saisĩ
259181,Like,vừa_vặn không nâu dày đỏ trắng đỏ cổ náu đẹp n...
259182,Like,đầu_shop hợp
259183,Like,đẹp đẹp hàng thích
...,...,...
1319014,Like,đóng_gói đẹp tốt
1319017,Like,tốt
1319036,Like,tuyệt_vời
1319045,Like,tốt


In [15]:
sub_data=spark.createDataFrame(data) 
sub_data.printSchema()

root
 |-- class: string (nullable = true)
 |-- pre_comment: string (nullable = true)



In [16]:
sub_data.show(3)

+-----+--------------------+
|class|         pre_comment|
+-----+--------------------+
| Like|          kiểm không|
| Like|cho_phép sơ_suất ...|
| Like|vừa_vặn không nâu...|
+-----+--------------------+
only showing top 3 rows



In [54]:
df_pre=sub_data

In [55]:
df_pre.show(3)

+-----+--------------------+
|class|         pre_comment|
+-----+--------------------+
| Like|          kiểm không|
| Like|cho_phép sơ_suất ...|
| Like|vừa_vặn không nâu...|
+-----+--------------------+
only showing top 3 rows



In [ ]:
# class_to_num = StringIndexer(inputCol='class',outputCol='label')
# #Fits a model to the input dataset with optional parameters.
# df_pre = class_to_num.fit(df_pre).transform(df_pre)
# df_pre.show(3)

In [56]:
tokenizer = Tokenizer(inputCol='pre_comment', outputCol='token_text')
# stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='token_text', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
class_to_num = StringIndexer(inputCol='class',outputCol='label')

In [57]:
assembler = VectorAssembler(inputCols=['tf_idf','label'], outputCol='features')

In [ ]:
df_pre.count()

In [58]:
data_prep_pipe = Pipeline(stages= [tokenizer,
                            count_vec,
                            idf,class_to_num,
                            assembler,lg])

In [ ]:
cleaner = data_prep_pipe.fit(df_pre)

In [ ]:
# cleaner.save(r"C:\Users\khaih\Downloads\Test_model\Data\Pipeline")

In [ ]:
clean_data= cleaner.transform(df_pre)

In [ ]:
clean_data.show(100)

In [ ]:
clean_data=clean_data.select(['label', 'features'])

In [22]:
(training, testing) = df_pre.randomSplit([0.7,0.3])

In [ ]:
training.groupby("class").count().show()

In [ ]:
testing.show(3)

In [ ]:
like_df = training.filter(col("class") == "Like")
neutral_df = training.filter(col("class") == "Neutral")
not_like_df = training.filter(col("class") == "Not_Like")
ratio_1 = int(like_df.count()/neutral_df.count())
ratio_2 = int(like_df.count()/not_like_df.count())
print("ratio like/neutral: {}".format(ratio_1))
print("ratio like/not_like: {}".format(ratio_2))

In [ ]:
# resample neutral
neutral_resample = range(ratio_1)

oversampled_neutral_df = neutral_df.withColumn("dummy", explode(array([lit(x) for x in neutral_resample]))).drop('dummy')

final_training_df = like_df.unionAll(oversampled_neutral_df)
final_training_df.show(10)

In [ ]:
final_training_df.groupBy("class").count().show()

In [ ]:
# resample not_like
not_like_resample = range(ratio_2)

oversampled_notlike_df = not_like_df.withColumn("dummy",explode(array([lit(x) for x in not_like_resample]))).drop('dummy')

final_training_df = final_training_df.unionAll(oversampled_notlike_df)
final_training_df.show(10)

In [ ]:
# (final_training_df, final_testing_df) = final_training_df.randomSplit([0.8,0.2], seed=42)

In [ ]:
final_training_df.groupBy("class").count().show()

### DecisionTree

In [ ]:
decisionTree=DecisionTreeClassifier()

In [ ]:
predictor_decisionTree = decisionTree.fit(training)

In [ ]:
test_results_decisionTree = predictor_decisionTree.transform(testing)

In [ ]:
test_results_decisionTree.groupby('label', 'prediction').count().show()

In [ ]:
test_results_decisionTree_rdd=test_results_decisionTree.select("label","prediction").rdd

In [ ]:
#predictionAndLabels = test_results_NaiveBayes.select("label","prediction")
from pyspark.mllib.evaluation import MulticlassMetrics


#predictionAndLabels = test_results_lg.select("label","prediction").rdd

# Instantiate metrics object
metrics_decisionTree = MulticlassMetrics(test_results_decisionTree_rdd)

# Statistics by class
labels = test_results_decisionTree_rdd.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics_decisionTree.precision(label)))
    print("Class %s recall = %s" % (label, metrics_decisionTree.recall(label)))
    print("Class %s F1 Measure = %s" % (label, metrics_decisionTree.fMeasure(label, beta=1.0)))

In [ ]:
acc_eval_decisionTree = MulticlassClassificationEvaluator()
acc_decisionTree = acc_eval_decisionTree.evaluate(test_results_decisionTree)
print("Accuracy of Model at  presicting like - dislike - neutral is: {}".format(acc_decisionTree))

In [ ]:
predictor_decisionTree.save("Project_1_DecisionTree_pyspark")

### NaiveBayes

In [ ]:
nb = NaiveBayes()

In [ ]:
predictor_NaiveBayes = nb.fit(final_training_df)

In [ ]:
test_results_NaiveBayes = predictor_NaiveBayes.transform(testing)

In [ ]:
test_results_NaiveBayes.groupby('label', 'prediction').count().show()

In [ ]:
acc_eval_NaiveBayes = MulticlassClassificationEvaluator()
acc_NaiveBayes = acc_eval_NaiveBayes.evaluate(test_results_NaiveBayes)
print("Accuracy of Model at presicting  like - dislike - neutral is: {}".format(acc_NaiveBayes))

In [ ]:
test_results_NaiveBayes_rdd=test_results_NaiveBayes.select("label","prediction").rdd

In [ ]:
#predictionAndLabels = test_results_NaiveBayes.select("label","prediction")
from pyspark.mllib.evaluation import MulticlassMetrics


#predictionAndLabels = test_results_lg.select("label","prediction").rdd

# Instantiate metrics object
metrics = MulticlassMetrics(test_results_NaiveBayes_rdd)

# Statistics by class
labels = test_results_NaiveBayes_rdd.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

In [ ]:
predictor_NaiveBayes.save("Project_1_NaiveBayes_pyspark")

### LogisticRegression

In [23]:
lg = LogisticRegression()

In [59]:
predictor_lg = data_prep_pipe.fit(training)

In [60]:
test_results_lg = predictor_lg.transform(testing)

In [61]:
test_results_lg.groupBy('label', 'prediction').count().show()

+-----+----------+------+
|label|prediction| count|
+-----+----------+------+
|  1.0|       1.0| 12000|
|  0.0|       1.0|   288|
|  2.0|       2.0|  9791|
|  1.0|       0.0|     9|
|  2.0|       1.0|    89|
|  1.0|       2.0|   734|
|  0.0|       0.0|129733|
|  0.0|       2.0|   172|
+-----+----------+------+



In [66]:
test_results_lg.tail(10)

[Row(class='Not_Like', pre_comment='ổn tệ giặt', token_text=['ổn', 'tệ', 'giặt'], c_vec=SparseVector(41725, {5: 1.0, 38: 1.0, 74: 1.0}), tf_idf=SparseVector(41725, {5: 2.1615, 38: 4.4976, 74: 5.3195}), label=1.0, features=SparseVector(41726, {5: 2.1615, 38: 4.4976, 74: 5.3195, 41725: 1.0}), rawPrediction=DenseVector([-12.1404, 11.1856, 0.9548]), probability=DenseVector([0.0, 1.0, 0.0]), prediction=1.0),
 Row(class='Not_Like', pre_comment='ổn tội_shop không_vưa cắt xấu hơi chán', token_text=['ổn', 'tội_shop', 'không_vưa', 'cắt', 'xấu', 'hơi', 'chán'], c_vec=SparseVector(41725, {5: 1.0, 18: 1.0, 41: 1.0, 77: 1.0, 121: 1.0, 4362: 1.0}), tf_idf=SparseVector(41725, {5: 2.1615, 18: 3.7702, 41: 4.5604, 77: 5.259, 121: 5.9048, 4362: 10.8392}), label=1.0, features=SparseVector(41726, {5: 2.1615, 18: 3.7702, 41: 4.5604, 77: 5.259, 121: 5.9048, 4362: 10.8392, 41725: 1.0}), rawPrediction=DenseVector([-8.8534, 10.9733, -2.1199]), probability=DenseVector([0.0, 1.0, 0.0]), prediction=1.0),
 Row(class

In [63]:
test_results_lg_rdd=test_results_lg.select("label","prediction").rdd

In [64]:
#predictionAndLabels = test_results_NaiveBayes.select("label","prediction")
from pyspark.mllib.evaluation import MulticlassMetrics


#predictionAndLabels = test_results_lg.select("label","prediction").rdd

# Instantiate metrics object
metrics = MulticlassMetrics(test_results_lg_rdd)

# Statistics by class
labels = test_results_lg_rdd.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

Class 0.0 precision = 0.9964667839284754
Class 0.0 recall = 0.9999306315610982
Class 0.0 F1 Measure = 0.9981957027718469
Class 1.0 precision = 0.9416934787726595
Class 1.0 recall = 0.9695402763189788
Class 1.0 F1 Measure = 0.9554140127388535
Class 2.0 precision = 0.9909919028340081
Class 2.0 recall = 0.9153033560811442
Class 2.0 F1 Measure = 0.9516450405792876


In [65]:
acc_eval_lg = MulticlassClassificationEvaluator()
acc_lg = acc_eval_lg.evaluate(test_results_lg)
print("Accuracy of model at predicting  like - dislike - neutral is: {}".format(acc_lg))

Accuracy of model at predicting  like - dislike - neutral is: 0.9916185929237098


In [ ]:
predictor_lg.save('Project_1_LogisticRegression_pyspark')

In [32]:
##LOAD EMOJICON
file = open('files/emojicon.txt', 'r', encoding="utf8")
emoji_lst = file.read().split('\n')
emoji_dict = {}
for line in emoji_lst:
    key, value = line.split('\t')
    emoji_dict[key] = str(value)
file.close()
#################
#LOAD TEENCODE
file = open('files/teencode.txt', 'r', encoding="utf8")
teen_lst = file.read().split('\n')
teen_dict = {}
for line in teen_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
file.close()
###############
#LOAD TRANSLATE ENGLISH -> VNMESE
file = open('files/english-vnmese.txt', 'r', encoding="utf8")
english_lst = file.read().split('\n')
english_dict = {}
for line in english_lst:
    key, value = line.split('\t')
    english_dict[key] = str(value)
file.close()
################
#LOAD wrong words
file = open('files/wrong-word.txt', 'r', encoding="utf8")
wrong_lst = file.read().split('\n')
file.close()
#################
#LOAD STOPWORDS
file = open('files/vietnamese-stopwords.txt', 'r', encoding="utf8")
stopwords_lst = file.read().split('\n')
file.close()

In [33]:
def process_text(text, emoji_dict, teen_dict, wrong_lst):
    document = text.lower()
    document = document.replace("’",'')
    document = regex.sub(r'\.+', ".", document)
    new_sentence =''
    for sentence in sent_tokenize(document):
        # if not(sentence.isascii()):
        ###### CONVERT EMOJICON
        sentence = ''.join(emoji_dict[word]+' ' if word in emoji_dict else word for word in list(sentence))
        ###### CONVERT TEENCODE
        sentence = ' '.join(teen_dict[word] if word in teen_dict else word for word in sentence.split())
        ###### DEL Punctuation & Numbers
        pattern = r'(?i)\b[a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]+\b'
        sentence = ' '.join(regex.findall(pattern,sentence))
        ###### DEL wrong words   
        sentence = ' '.join('' if word in wrong_lst else word for word in sentence.split())
        new_sentence = new_sentence+ sentence + '. '                    
    document = new_sentence  
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [34]:
# Chuẩn hóa unicode tiếng việt
def loaddicchar():
    uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
    unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
 
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def convert_unicode(txt):
    dicchar = loaddicchar()
    return regex.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [35]:
# có thể bổ sung thêm các từ: chẳng, chả...
def process_special_word(text):
    new_text = ''
    text_lst = text.split()
    i= 0
    if 'không' in text_lst:
        while i <= len(text_lst) - 1:
            word = text_lst[i]
#             print(word)
#             print(i)
            if  word == 'không':
                next_idx = i+1
                if next_idx <= len(text_lst) -1:
                    word = word +'_'+ text_lst[next_idx]
#                     print(word)
                i= next_idx + 1
            else:
                i = i+1
            new_text = new_text + word + ' '
    if 'tạm' in text_lst:
        while i <= len(text_lst) - 1:
            word = text_lst[i]
#             print(word)
#             print(i)
            if  word == 'tạm':
                next_idx = i+1
                if next_idx <= len(text_lst) -1:
                    word = word +'_'+ text_lst[next_idx]
#                     print(word)
                i= next_idx + 1
            else:
                i = i+1
            new_text = new_text + word + ' '
    if 'hơi' in text_lst:
        while i <= len(text_lst) - 1:
            word = text_lst[i]
#             print(word)
#             print(i)
            if  word == 'hơi':
                next_idx = i+1
                if next_idx <= len(text_lst) -1:
                    word = word +'_'+ text_lst[next_idx]
#                     print(word)
                i= next_idx + 1
            else:
                i = i+1
            new_text = new_text + word + ' '
            #print(new_text)
    else:
        new_text = text
    return new_text.strip()

In [36]:
def process_postag_thesea(text):
    new_document = ''
    for sentence in sent_tokenize(text):
        sentence = sentence.replace('.','')
        ###### POS tag
        lst_word_type = ['A','AB','V','VB','VY','R','M','N','NP']
        #lst_word_type = ['A','AB','V','VB','VY','R']
        sentence = ' '.join( word[0] if word[1].upper() in lst_word_type else '' for word in pos_tag(process_special_word(word_tokenize(sentence, format="text"))))
        new_document = new_document + sentence + ' '
    ###### DEL excess blank space
    new_document = regex.sub(r'\s+', ' ', new_document).strip()
    return new_document

In [37]:
def remove_stopword(text, stopwords):
    ###### REMOVE stop words
    document = ' '.join('' if word in stopwords else word for word in text.split())
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [ ]:
# from pyspark.ml.classification import LogisticRegressionModel
# model_1 = LogisticRegressionModel.load('Project_1_LogisticRegression_pyspark_2')

In [87]:
str_1="Hàng Shopee Mall chuẩn code. Chất lỏng, lớp finish ẩm mượt, nâng tông tuy nhiên thì với bạn nào có oily-skin như mình sản phẩm có vẻ phù hợp 80% (thêm 1 lớp phấn phủ sẽ Okiela hơn nha)"

In [68]:
def xu_li_text(text):
    document=  process_text(text,emoji_dict,teen_dict,wrong_lst)
    document = convert_unicode(document)
    document = process_postag_thesea(document)
    document = remove_stopword(document, stopwords_lst)
    return document

In [88]:
from pyspark.sql import Row
from pyspark.sql.types import StringType
sentences = spark.createDataFrame([
    Row(pre_comment=xu_li_text(str_1),label=5)])
sentences.show(truncate=False)

+--------------------------------------------------------------------------------------------+-----+
|pre_comment                                                                                 |label|
+--------------------------------------------------------------------------------------------+-----+
|hàng shopee chuẩn chất_lỏng lớp finish ẩm mượt nâng tông nhiên oily skin lớp phấn phủ okiela|5    |
+--------------------------------------------------------------------------------------------+-----+



In [ ]:
sentences = cleaner.transform(sentences)

In [ ]:
sentences.show()

In [ ]:
prec_1=predictor_lg.transform(sentences)

In [ ]:
prec_1.select("pre_comment","prediction").show()

In [ ]:
if prec_1.select("prediction").collect()[0][0]==1.0:
    print("Like")

In [ ]:
0:like
2: dislike
1: neutral

In [ ]:
from pyspark.ml import PipelineModel
pipeline_test=PipelineModel.load('Project_1_Pipeline_pyspark')

In [ ]:
test_text = pipeline_test.transform(sentences)

In [89]:
# new_sentence = "Thật sự không thích sản phẩm này"
# new_df = spark.createDataFrame([(new_sentence,)], ["pre_comment"])
prediction = predictor_lg.transform(sentences)

In [90]:
prediction.show()

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|         pre_comment|label|          token_text|               c_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|hàng shopee chuẩn...|    5|[hàng, shopee, ch...|(41725,[35,49,149...|(41725,[35,49,149...|(41726,[35,49,149...|[-198.19884419223...|[2.92609995502389...|       2.0|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+



### Nhận xét:
- Các model chạy bằng thư viện pyspark đều cho ra kết quả khá tốt (83 - 99%).
- Xem xét recall, f1 Measure, precision ta thấy model NaiveBayes chưa thực sự tốt. Ngược lại model LogisticRegression cho ra accuracy lẫn recall, f1 Measure, precision đều khá cao.

=> Model LogisticRegression cho ra kết quả tốt nhất và có thể áp dụng model này.




- Do cấu hình máy yếu nên em không thể chạy model RandomForestClassifier và một số model còn lại.
- Các model GBT, SVC chỉ hỗ trợ chạy được 2 classes không thể chạy 3 classes trở lên

### RandomForestClassifier

In [ ]:
# rfc=RandomForestClassifier()

In [ ]:
# predictor_rfc = rfc.fit(final_training_df)

In [ ]:
# test_results_rfc = predictor_rfc.transform(testing)

In [ ]:
# test_results_rfc.groupby('label', 'prediction').count().show()

In [ ]:
# acc_eval_rfc = MulticlassClassificationEvaluator()
# acc_rfc = acc_eval_rfc.evaluate(test_results_rfc)
# print("Accuracy of Model at  presicting like - dislike - neutral is: {}".format(acc_rfc))

### GBT

In [ ]:
# gbt= GBTClassifier(maxIter=10)

In [ ]:
# predictor_gbt = gbt.fit(training)

In [ ]:
# test_results_gbt = predictor_gbt.transform(testing)

In [ ]:
# test_results_gbt.groupby('label', 'prediction').count().show()

In [ ]:
# acc_eval_gbt = MulticlassClassificationEvaluator()
# acc_gbt = acc_eval_gbt.evaluate(test_results_gbt)
# print("Accuracy of Model at  presicting spam ham is: {}".format(acc_gbt))

### SVC

In [ ]:
# lsvc = LinearSVC(maxIter=10, regParam=0.1)

In [ ]:
# predictor_lsvc = lsvc.fit(training)

In [ ]:
# test_results_lsvc = predictor_lsvc.transform(testing)

In [ ]:
# test_results_lsvc.groupBy('label', 'prediction').count().show()

In [ ]:
# acc_eval_lsvc = MulticlassClassificationEvaluator()
# acc_lsvc = acc_eval_lsvc.evaluate(test_results_lg)
# print("Accuracy of model at predicting  like - dislike - neutral is: {}".format(acc_lsvc))

### One and Rest

In [ ]:
# from pyspark.ml.classification import OneVsRest
# ovr = OneVsRest(classifier=GBTClassifier())

In [ ]:
# predictor_ovr = ovr.fit(training)

In [ ]:
# test_results_ovr = predictor_ovr.transform(testing)

In [ ]:
# test_results_ovr.count()

In [ ]:
# test_results_ovr.groupBy('label', 'prediction').count().show()

In [ ]:
# acc_eval = MulticlassClassificationEvaluator()
# acc_1 = acc_eval.evaluate(test_results_ovr)
# print("Accuracy of model at predicting: {}".format(acc_1))

In [ ]:
print(1)